## Decision Analytics Final Project
### Question: What is the best combination of tourist attractions in NYC that a travel agency can put together in order to provide the best tour experience? 
#### Team Members: Megan, Jhanvi, Hannah, Kaia

In [2]:
import numpy as np
from gurobipy import *
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
survey_table = pd.read_csv("survey_final_table.csv")
nyc = pd.read_csv("nyc attractions.csv")

In [4]:
survey_table.head()

,Unnamed: 0,Group,Proba,Nature,Museum,Family,Activity,Walking,Theatre,Landmarks
0,0,3.0,0.25,3.800000,2.000000,3.400000,4.400000,3.000000,2.400000,2.800000
1,1,2.0,0.20,1.000000,3.750000,3.000000,1.250000,2.250000,3.500000,3.000000
2,2,1.0,0.30,4.333333,3.666667,3.166667,1.833333,4.166667,3.333333,1.333333
3,3,0.0,0.25,5.000000,3.200000,1.800000,4.200000,2.600000,3.200000,1.400000


In [7]:
nyc.head()

,AttractionID,AttractionName,CostPerVisit,Nature,Museum,Landmark,FamilyFriendly,Active,Arts,Unnamed: 9,Level,Meaning
0,1,Central Park,0.0,3,2,3,3,3,2,NaN,1.0,No relevance
1,2,The National 9/11 Memorial & Museum,33.0,1,3,2,2,1,2,NaN,2.0,Some relevance
2,3,The Metropolitan Museum of Art,30.0,1,3,2,2,1,3,NaN,3.0,High relevance
3,4,Empire State Building,79.0,1,1,3,3,1,1,NaN,NaN,NaN
4,5,The High Line,0.0,3,1,3,3,3,1,NaN,NaN,NaN


In [12]:
columns_drop = ['Unnamed: 9' ,  'Level ', 'Meaning']
nyc = nyc.drop(columns=columns_drop, inplace=True)

KeyError: "['Unnamed: 9', 'Level ', 'Meaning'] not found in axis"

In [13]:
nyc.head()

,AttractionID,AttractionName,CostPerVisit,Nature,Museum,Landmark,FamilyFriendly,Active,Arts
0,1,Central Park,0.0,3,2,3,3,3,2
1,2,The National 9/11 Memorial & Museum,33.0,1,3,2,2,1,2
2,3,The Metropolitan Museum of Art,30.0,1,3,2,2,1,3
3,4,Empire State Building,79.0,1,1,3,3,1,1
4,5,The High Line,0.0,3,1,3,3,3,1


In [30]:
nyc.columns

Index(['AttractionID', 'AttractionName', 'CostPerVisit', 'Nature', 'Museum ',
       'Landmark', 'FamilyFriendly', 'Active', 'Arts'],
      dtype='object')

In [32]:
nyc['avg'] = nyc[['Nature', 'Museum ',
       'Landmark', 'FamilyFriendly', 'Active', 'Arts']].mean(axis=1)

In [33]:
nyc.head()

,AttractionID,AttractionName,CostPerVisit,Nature,Museum,Landmark,FamilyFriendly,Active,Arts,avg
0,1,Central Park,0.0,3,2,3,3,3,2,2.666667
1,2,The National 9/11 Memorial & Museum,33.0,1,3,2,2,1,2,1.833333
2,3,The Metropolitan Museum of Art,30.0,1,3,2,2,1,3,2.000000
3,4,Empire State Building,79.0,1,1,3,3,1,1,1.666667
4,5,The High Line,0.0,3,1,3,3,3,1,2.333333


In [20]:
attractions1 = nyc.to_dict()

In [22]:
attractions1

{'AttractionID': {0: 1,
  1: 2,
  2: 3,
  3: 4,
  4: 5,
  5: 6,
  6: 7,
  7: 8,
  8: 9,
  9: 10,
  10: 11,
  11: 12,
  12: 13,
  13: 14,
  14: 15,
  15: 16,
  16: 17,
  17: 18,
  18: 19,
  19: 20,
  20: 21,
  21: 22,
  22: 23,
  23: 24},
 'AttractionName': {0: 'Central Park',
  1: 'The National 9/11 Memorial & Museum',
  2: 'The Metropolitan Museum of Art',
  3: 'Empire State Building',
  4: 'The High Line',
  5: 'Top of the Rock',
  6: 'Statue of Liberty',
  7: 'Times Square',
  8: 'Broadway Show',
  9: 'Grand Central Terminal',
  10: 'Brooklyn Bridge',
  11: 'American Musuem of Natural History',
  12: 'Staten Island Ferry',
  13: 'SUMMIT One Vanderbilt',
  14: 'Chelsea Market',
  15: 'NY Public Library',
  16: 'Rockerfeller Centre',
  17: 'MoMA',
  18: 'One World Observatory',
  19: 'Bryant Park',
  20: 'Intrepid Museum ',
  21: 'Shopping on Fifth Avenue',
  22: 'Shopping at SoHo',
  23: 'Roosevelt Island'},
 'CostPerVisit': {0: 0.0,
  1: 33.0,
  2: 30.0,
  3: 79.0,
  4: 0.0,
  5: 40

In [36]:

# Parameters (example data, update based on your specific inputs)
#attractions = [{"id": 1, "cost": 0, "happiness": 2, "accessibility": 1, "family_friendly": 1},  # Example: Central Park {"id": 2, "cost": 33, "happiness": 3, "accessibility": 0, "family_friendly": 1},  # Example: 9/11 Memorial
    # Add all attractions here with relevant attributes]

budget = 1000  # Total budget constraint
max_duration = 8  # Maximum number of hours available for the tour
travel_time = np.random.randint(1, 3, size=(len(attractions1), len(attractions1)))  # Travel time matrix (example)
demographic_weights = [0.25, 0.20, 0.30, 0.25]  # Example weights for demographic groups
scenario_probabilities = [0.5, 0.5]  # Probabilities for scenarios (adjust as needed)

# Initialize Model
model = Model("NYC_Tour_Optimization")

# Decision Variables
A = model.addVars(len(attractions1), vtype=GRB.BINARY, name="A")  # Binary decision for each attraction

# Objective Function
happiness_scores = [
    demographic_weights[j] * attractions1[i]["avg"]
    for i in range(len(attractions1))
    for j in range(len(demographic_weights))
    
]

model.setObjective(quicksum(A[i] * happiness_scores[i] for i in range(len(attractions1))), GRB.MAXIMIZE)

# Constraints

# Budget Constraint
model.addConstr(quicksum(A[i] * attractions1[i]["cost"] for i in range(len(attractions1))) <= budget, "Budget")

# Total Duration Constraint (tour cannot exceed available time)
attraction_durations = [1.5 for _ in range(len(attractions1))]  # Example durations for each attraction in hours
model.addConstr(
    quicksum(A[i] * attraction_durations[i] for i in range(len(attractions1))) <= max_duration, "Duration"
)

# Nature Constraint (at least 2 of selected attractions must be Nature-related)
model.addConstr(
    quicksum(A[i] * attractions1[i]["Nature"] for i in range(len(attractions1)))
    >= 2/23 * quicksum(A[i] for i in range(len(attractions1))),
    "Accessibility"
)

# Family-Friendliness Constraint (at least 50% of selected attractions must be family-friendly)
model.addConstr(
    quicksum(A[i] * attractions1[i]["FamilyFriendly"] for i in range(len(attractions1)))
    >= 0.5 * quicksum(A[i] for i in range(len(attractions1))),
    "Family_Friendly"
)

#Arts related constraint
model.addConstr(
    quicksum(A[i] * attractions1[i]["Arts"] for i in range(len(attractions1)))
    >= 3/23 * quicksum(A[i] for i in range(len(attractions1))),
    "Accessibility"
)

#Museum

model.addConstr(
    quicksum(A[i] * attractions1[i]["Museum"] for i in range(len(attractions1)))
    >= 1/23 * quicksum(A[i] for i in range(len(attractions1))),
    "Accessibility"
)

# Active
model.addConstr(
    quicksum(A[i] * attractions1[i]["Active"] for i in range(len(attractions1)))
    >= 1/23 * quicksum(A[i] for i in range(len(attractions1))),
    "Accessibility"
)

#Landmark 

model.addConstr(
    quicksum(A[i] * attractions1[i]["Landmark"] for i in range(len(attractions1)))
    >= 5/23 * quicksum(A[i] for i in range(len(attractions1))),
    "Accessibility"
)

# Pairwise Constraints (if one attraction is selected, another must also be selected)
model.addConstr(A[0] + A[1] <= 1, "Pairwise_Constraint")  # Example: Cannot select both Central Park and 9/11 Memorial

# Optimize
model.optimize()

# Print Results
if model.status == GRB.OPTIMAL:
    selected_attractions = [i for i in range(len(attractions1)) if A[i].X > 0.5]
    print("Optimal Attractions:", selected_attractions)
    print("Maximum Happiness:", model.objVal)
else:
    print("No optimal solution found.")


KeyError: 0

In [38]:
import numpy as np
from gurobipy import *
import pandas as pd

# Load data
survey_table = pd.read_csv("survey_final_table.csv")
nyc = pd.read_csv("nyc attractions.csv")

# Clean and preprocess data
columns_drop = ['Unnamed: 9', 'Level', 'Meaning']
nyc = nyc.drop(columns=[col for col in columns_drop if col in nyc.columns])

# Add average column
nyc['avg'] = nyc[['Nature', 'Museum ', 'Landmark', 'FamilyFriendly', 'Active', 'Arts']].mean(axis=1)

# Convert to dictionary for optimization
attractions1 = nyc.set_index('AttractionID').to_dict('index')

# Parameters
budget = 1000  # Total budget constraint
max_duration = 8  # Maximum number of hours available for the tour
demographic_weights = [0.25, 0.20, 0.30, 0.25]  # Example weights for demographic groups

# Initialize Model
model = Model("NYC_Tour_Optimization")

# Decision Variables
A = model.addVars(len(attractions1), vtype=GRB.BINARY, name="A")  # Binary decision for each attraction

# Objective Function
happiness_scores = [
    demographic_weights[j] * attractions1[i + 1]["avg"]  # Access "avg" correctly
    for i in range(len(attractions1))
    for j in range(len(demographic_weights))
]

# Align scores with the attraction index
adjusted_scores = [
    sum(demographic_weights[j] * attractions1[i + 1]["avg"] for j in range(len(demographic_weights)))
    for i in range(len(attractions1))
]

model.setObjective(quicksum(A[i] * adjusted_scores[i] for i in range(len(attractions1))), GRB.MAXIMIZE)

# Constraints

# Budget Constraint
model.addConstr(
    quicksum(A[i] * attractions1[i + 1]["CostPerVisit"] for i in range(len(attractions1))) <= budget, "Budget"
)

# Total Duration Constraint (tour cannot exceed available time)
attraction_durations = [1.5 for _ in range(len(attractions1))]  # Example durations for each attraction in hours
model.addConstr(
    quicksum(A[i] * attraction_durations[i] for i in range(len(attractions1))) <= max_duration, "Duration"
)

# Constraints for specific categories
categories = ["Nature", "Museum ", "Landmark", "FamilyFriendly", "Active", "Arts"]
category_min_ratios = [2 / 23, 1 / 23, 5 / 23, 0.5, 1 / 23, 3 / 23]  # Example minimum ratios

for category, min_ratio in zip(categories, category_min_ratios):
    model.addConstr(
        quicksum(A[i] * attractions1[i + 1][category] for i in range(len(attractions1))) 
        >= min_ratio * quicksum(A[i] for i in range(len(attractions1))), f"{category}_Constraint"
    )

# Pairwise Constraints (e.g., cannot select both Central Park and 9/11 Memorial)
model.addConstr(A[0] + A[1] <= 1, "Pairwise_Constraint")

# Optimize
model.optimize()

# Print Results
if model.status == GRB.OPTIMAL:
    selected_attractions = [i + 1 for i in range(len(attractions1)) if A[i].X > 0.5]
    print("Optimal Attractions:", selected_attractions)
    print("Maximum Happiness:", model.objVal)
else:
    print("No optimal solution found.")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 24.1.0 24B91)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 9 rows, 24 columns and 180 nonzeros
Model fingerprint: 0xb4e260f3
Variable types: 0 continuous, 24 integer (24 binary)
Coefficient statistics:
  Matrix range     [8e-01, 1e+02]
  Objective range  [2e+00, 3e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Found heuristic solution: objective 10.3333333
Presolve removed 9 rows and 24 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 11.6667 10.3333 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.166666666667e+01, best bound 1.166666666667e+01, gap 0.0000%
Optimal Attractions: [1, 5, 12, 20, 24]
Maximum Happiness: 11.666666666666666
